\section*{Методы решения ОДУ}

\subsection*{Постановка задач}
1. Построить экстраполяционный метод Адамса пятого порядка (дать представление в двух формах).
2. Определить порядок точности метода
$$\begin{cases}
y_{j+1}=y_j+\frac{\tau}{6}(f_j+4f_{j+\frac{1}{2}}+f_{j+1}),\\
y_j=y_{j+1}-\tau f_{j+\frac{1}{2}}, \\
y_{j+\frac{1}{2}}=y_{j+1}-\frac{\tau}{2}f_{j+1}.
\end{cases}$$
3. Найти интервал устойчивости метода из п. 1;
4. С заданной точность $\epsilon=10^{-4}$ найти решение задачи Коши с помощью: а) метода последовательного повышения порядка точности четвертого порядка; б) явного метода Рунге-Кутта третьего порядка; в) интерполяционого метода Адамса третьнго порядка
$$\begin{cases}
u'=\frac{1}{5}\begin{pmatrix}-104 & -198 \\
-198 & -401 \end{pmatrix} u, t \in [0; 5], \\
u(0)=\begin{pmatrix}5 \\ 10 \end{pmatrix}.
\end{cases}$$

\subsection*{Задача 1}

Экстраполяционным методом Адамса называется метод, получаемый по формуле
$$y_{j+1} = y_j + h\sum_{i=0}^{k}A_i f(x_{j-i}, y_{j-i}),$$
где коэффициенты вычисляются через интегралы $$A_i = \dfrac{(-1)^i}{i! (k-i)!}\int\limits_0^1 \dfrac{\alpha(\alpha+1)\ldots (\alpha+k)}{\alpha+i}d\alpha.$$
В случае метода пятого порядка, нам необходимо посчитать интегралы $A_i$ для всех $i=\overline{0,k}.$ Поскольку нас интересует метод пятого порядка, то выберем $k=4$

$$i=0: A_0 = \frac{1}{24}\int_0^1(\alpha+1)(\alpha+2)(\alpha+3)(\alpha+4)d\alpha;$$
$$i=1: A_1 = -\frac{1}{6}\int_0^1\alpha(\alpha+2)(\alpha+3)(\alpha+4)d\alpha;$$
$$i=2: A_2 = \frac{1}{4}\int_0^1\alpha(\alpha+1)(\alpha+3)(\alpha+4)d\alpha;$$
$$i=3: A_3 = -\frac{1}{6}\int_0^1\alpha(\alpha+1)(\alpha+2)(\alpha+4)d\alpha;$$
$$i=4: A_4 = \frac{1}{24}\int_0^1\alpha(\alpha+1)(\alpha+2)(\alpha+3)d\alpha;$$
Для вычисления данных интегралов воспользуемся помощью инструментов Python

In [16]:
from scipy import integrate
import math

k = 4

def f(x, k, n=4):
    res = 1
    
    for i in range(n + 1):
       res *= (x + i)
    
    return res / (x + k)

for i in range(k + 1):
    print(f'Интеграл {i} равен: {((-1)**i) / (math.factorial(i) * math.factorial(k - i)) * integrate.quad(f, 0, 1, args=(i,))[0]}')

Интеграл 0 равен: 2.6402777777777775
Интеграл 1 равен: -3.852777777777778
Интеграл 2 равен: 3.6333333333333337
Интеграл 3 равен: -1.7694444444444442
Интеграл 4 равен: 0.3486111111111112


Таким образом, получаем $A_0 \approx 2.64, A_1 \approx -3.85, A_2 \approx 3.63, A_3 \approx -1.77, A_4 \approx 0.35$. Тогда метод Адамса пятого порядка будет иметь вид

$$y_{j+1} = y_j + h(2.64f_j - 3.85 f_{j-1} + 3.63 f_{j-2} - 1.77 f_{j-3} + 0.35 f_{j-4}).$$

Получим представление метода Адамса пятого порядка через конечные разности, формула для этого метода имеет вид 
$$y_{j+1} = y_j + h\sum_{i=0}^{k}C_i \Delta ^i f_{j-i},$$
	 $$C_i = \dfrac{1}{i!}\int\limits_0^1 \alpha(\alpha+1)\ldots(\alpha+i-1)d\alpha.$$
Аналогичным образом, найдем все $C_i$ для $i=\overline{0,k},$ $k$ выбираем то же

$$i=0: C_0 = 0;$$
$$i=1: C_1 = \int_0^1\alpha d\alpha = 1;$$
$$i=2: C_2 = \frac{1}{2!}\int_0^1\alpha(\alpha+1)d\alpha = \frac{1}{2}\int_0^1(\alpha^2+\alpha) d\alpha = \frac{5}{12};$$
$$i=3: C_3 = \frac{1}{3!}\int_0^1\alpha(\alpha+1)(\alpha+2)d\alpha=\frac{1}{6}\int_0^1 (\alpha^3+3\alpha^2+2\alpha) d\alpha = \frac{3}{8};$$
$$i=4: C_4 = \frac{1}{4!}\int_0^1\alpha(\alpha+1)(\alpha+2)(\alpha+3) d\alpha = \frac{1}{24}\int_0^1(\alpha^4+6\alpha^3+11\alpha^2+6\alpha) d\alpha = \frac{1}{24}\Bigl(\frac{1}{5}+\frac{3}{2}+\frac{11}{3}+3\Bigr) = \frac{251}{720};$$

Таким образом, можем составить экстраполяционый метод Адамса пятого порядка через конечные разности
$$y_{j+1}=y_j+h\Bigl( \Delta^1 f_{j-1} + \frac{5}{12} \Delta^2 f_{j-2} + \frac{3}{8} \Delta^3 f_{j-3} + \frac{251}{720} \Delta^4 f_{j-4}\Bigr).$$

\subsection*{Задача 2}

Для определения порядка точности нашего метода первым делом оценим локальные погрешности, определяемые следующим образом
$$r(t_j, \tau)=u(t_{j+1})-F\big(u(t_{j-q}),\ldots, u(t_j), u(t_{j+1}),\ldots,u(t_{j+s})\big).\eqno(2.1)$$
Для удобства переопределим $u(t_j)=u_j.$
Тогда, переходя к нашему методу, будем поочередно рассматривать уравнения от меньшего порядка к большему. Приступим к рассмотрению третьего уравнения
$$y_{j+\frac{1}{2}}=y_{j+1}-\frac{\tau}{2}f_{j+1}$$
Тогда, применяя формулу (2.1), получим
$$r(t_j,\tau)=u_{j+\frac{1}{2}}-u_{j+1}+\frac{\tau}{2}f_{j+1}$$
Для оценки погрешности разложим функции по степеням $\tau$
$$u_{j+\frac{1}{2}}\approx u(t_j+\frac{\tau}{2}) = u_j + \frac{\tau}{2\cdot 1!}u'_j+\frac{\tau^2}{4 \cdot 2!}u''_j+O(\tau^3);$$
$$u_{j+1} \approx u(t_j+\tau)=u_j+\frac{\tau}{1!}u'_j+\frac{\tau^2}{2!}u''_j+O(\tau^3);$$
Для того, чтобы разложить $f_{j+1}$ по степеням $\tau$ воспользуемся тем, что мы решаем задачу Коши вида
$$f(x,u(x)) = u'(x),\  u|_{x=x_0}=u_0.$$
Тогда
$$f_{j+1} \approx f(t_j+\tau, u(t_j)) = u'(t_j+\tau)=u'_j+\frac{\tau}{1!}u''_j+\frac{\tau^2}{2!}u'''_j+O(\tau^3).$$
Подставим полученные разложения в рассматриваемое уравнение

$$r(t_j,\tau)\approx u_j + \frac{\tau}{2}u'_j+\frac{\tau^2}{8}u''_j -u_j-\tau u'_j-\frac{\tau^2}{2}u''_j+\frac{\tau}{2}\Bigl( u'_j+\tau u''_j+\frac{\tau^2}{2}u'''_j \Bigr) + O(\tau^3)=\frac{\tau}{2}u'_j-\tau u'_j + \frac{\tau}{2}u'_j+\frac{\tau^2}{8}u''_j-\frac{\tau^2}{2}u''_j+\frac{\tau^2}{2}u''_j+\frac{\tau^3}{4}u'''_j+O(\tau^3)=\frac{\tau^2}{8}u''_j+\frac{\tau^3}{4}u'''_j+O(\tau^3)=O(\tau^2).$$
Таким образом, локальная погрешность равна $r(t_j,\tau)=O(\tau^2).$ Перейдем к следующему уравнению
$$y_j=y_{j+1}-\tau f_{j+\frac{1}{2}}.$$
Проделаем ту же процедуру
$$r(t_j,\tau)=u_{j}-u_{j+1}+\tau f_{j+\frac{1}{2}}$$
Приступим к разложению по степеням $\tau$. Поскольку мы уже раскладывали $u_{j+1},$ а разложение $u_j$ приведет нас к $u_j$, то сразу перейдем к разложению третьего слагаемого
$$f_{j+\frac{1}{2}}\approx u'(t_j+\frac{\tau}{2}) = u'_j+\frac{\tau}{2}u''_j+\frac{\tau^2}{8}u'''_j+O(\tau^3).$$
Подставим 

$$r(t_j, \tau) \approx u_j - u_j - \tau u'_j - \frac{\tau^2}{2}u''_j+\tau \Bigl( u'_j+\frac{\tau}{2}u''_j+\frac{\tau^2}{8}u'''_j \Bigr) + O(\tau^3)=\frac{\tau^3}{8}u'''_j + O(\tau^3) = O(\tau^3).$$
Таким образом, локальная погрешность в этом случае равна $r(t_j, \tau) = O(\tau^3)$. Перейдем к последнему(первому) уравнению
$$r(t_j, \tau) = u_{j+1}-u_j-\frac{\tau}{6}(f_j+4f_{j+\frac{1}{2}}+f_{j+1}).$$
Аналогично предыдущему уравнению, сразу перейдем к подстановке, учитвая, что $f_j \approx u'_j$
$$r(t_j, \tau) \approx u_j+\tau u'_j+\frac{\tau^2}{2}u''_j - u_j - \frac{\tau}{6}\Bigl(u'_j + 4 \Bigl(u'_j+\frac{\tau}{2}u''_j+\frac{\tau^2}{8}u'''_j\Bigr) + u'_j+\tau u''_j+\frac{\tau^2}{2}u'''_j\Bigr) + O(\tau^3)=O(\tau^3).$$
Таким образом, мы получили, что у нас сократились все слагаемые, поэтому нам необходимо увеличить количество членов разложения, скажем, до третьей степени, сразу выполним подстановку с учетом полученных результатов
$$r(t_j, \tau) \approx \frac{\tau^3}{6}u'''_j-\frac{\tau}{6}\Bigl( \frac{4\tau^3}{12}u^{(4)}_j+\frac{\tau^3}{6}u^{(4)}_j\Bigr) + O(\tau^4) = O(\tau^3).$$
Таким образом, локальная погрешность равна $r(t_j, \tau)=O(\tau^3),$ найдем глобальную погрешность нашего метода
$$\psi(t_j, \tau) = \dfrac{O(\tau^3)}{\tau} = O(\tau^2).$$
Отсюда следует, что рассматриваемый метод является методом второго порядка точности.